In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import pandas as pd

In [ ]:
def append_college_stats(rows, year):
    player_stats = []
    for i in range(len(rows)):
        player = [td.getText() for td in rows[i].findAll('td')]
        if player == []:
            continue
        td_list = rows[i].findAll('td')
    for i in range(len(rows)):
        player = [td.getText() for td in rows[i].findAll('td')]
        if player == []:
            continue
        td_list = rows[i].findAll('td')
        for td in td_list:  
            if td['data-stat'] == "player":
                button = td.findAll('a')
                
                if button != []:
                    link = button[0]['href']
                    player_url = "https://www.basketball-reference.com{}".format(link)
                    player_html = urlopen(player_url)
                    player_soup = BeautifulSoup(player_html)
                    college_stats = player_soup.findAll(id="all_all_college_stats")
                    
                    player_height_temp = None
                    if player_soup.findAll("span",{'itemprop': 'height'})[0]:
                        player_height_temp = player_soup.findAll("span",{'itemprop': 'height'})[0].text
                    
                    player_height = None
                    if (int(player_height_temp.split('-')[0])+(int(player_height_temp.split('-')[1]))):
                        player_height = int(player_height_temp.split('-')[0])+(int(player_height_temp.split('-')[1])/12)
                    
                    player_weight = None
                    if(player_soup.findAll("span",{'itemprop':'weight'})[0]):                                        
                        player_weight = player_soup.findAll("span",{'itemprop':'weight'})[0].text
                    
                    birth_date = None
                    if(player_soup.findAll("div",{"itemtype":"https://schema.org/Person"})[0].findAll("span",{"id":"necro-birth"})[0]):
                        birth_date = player_soup.findAll("div",{"itemtype":"https://schema.org/Person"})[0].findAll("span",{"id":"necro-birth"})[0].text.replace('\n','').strip()
                    
                    birth_place = None
                    if(player_soup.findAll("div",{"itemtype":"https://schema.org/Person"})[0].find("span",{"itemprop":"birthPlace"}).a):
                        birth_place = player_soup.findAll("div",{"itemtype":"https://schema.org/Person"})[0].find("span",{"itemprop":"birthPlace"}).a.text                    
                    
                    for each in player_soup.findAll('p'):
                        if 'Position:' in each.text:
                            player_position = each.text.replace('\n','').strip().split(' ')[4]
                    if college_stats != []:
                        for comments in college_stats[0].findAll(text=lambda text:isinstance(text, Comment)):
                            x = comments.extract()
                            comment_soup = BeautifulSoup(x , 'lxml')
                            tfoot = comment_soup.findAll('tfoot')
                            td = tfoot[0].findAll('td')
                            for i in range(2, len(td)):
                                player.append(td[i].getText())
                            player.append(player_height)
                            player.append(player_weight)
                            player.append(birth_date)
                            player.append(birth_place)
                            player.append(player_position)
                            player.append(year)
        player_stats.append(player)
        print('PROCESSING PLAYER {}'.format(player_url))
    return player_stats
        
    

In [ ]:
# Defining range of draft classes
start_year = 1990
end_year = 2020
college_headers = ['college_G',
                   'college_MP',
                   'college_FG',
                    'college_FGA',
                    'college_3P',
                    'college_3PA',
                    'college_FT',
                    'college_FTA',
                    'college_ORB',
                    'college_TRB',
                    'college_AST',
                    'college_STL',
                    'college_BLK',
                    'college_TOV',
                    'college_PF',
                    'college_PTS',
                    'college_FG%',
                    'college_3P%',
                    'college_FT%',
                    'college_MP',
                    'college_PTS%',
                    'college_TRB',
                    'college_AST',
                   'height',
                   'weight',
                   'date_of_birth',
                   'place_of_birth',
                   'position',
                   'year'
                  ]

# Initialize .csv writer
writer = pd.ExcelWriter('nba_draft.xlsx', engine='xlsxwriter')

# Push data to .csv file (1 sheet per draft class)
for i in range(start_year, end_year + 1):
    url = "https://www.basketball-reference.com/draft/NBA_{}.html".format(i)
    html = urlopen(url)
    soup = BeautifulSoup(html)
    rows = soup.findAll('tr', limit=2)
    headers = [th.getText() for th in rows[1].findAll('th')]
    headers = headers[1:]
    headers = headers + college_headers    
    rows = soup.findAll('tr')[2:]
    player_stats = append_college_stats(rows, i)
    stats = pd.DataFrame(player_stats, columns = headers)
    stats = stats.dropna()
    data = stats.head(60)
    stats.to_excel(writer, sheet_name='draft_data_{}'.format(i), index = False)
    
# Save .csv data
print('Scraping Completed!')
writer.save()

PROCESSING PLAYER https://www.basketball-reference.com/players/c/colemde01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/p/paytoga01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/a/abdulma02.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/scottde01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/g/gillke01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/spencfe01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/simmoli01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/k/kimblbo01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/b/burtowi01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/r/robinru01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/h/hillty01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/k/kesslal01.html
PROCESSING PLAYER https://www.basketball-r

PROCESSING PLAYER https://www.basketball-reference.com/players/t/tabakza01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/t/tabakza01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/t/tabakza01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/t/tabakza01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/o/onealsh01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/m/mournal01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/l/laettch01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/j/jacksji01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/e/ellisla01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/g/guglito01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/w/williwa02.html
PROCESSING PLAYER https://www.basketball-reference.com/players/d/dayto01.html
PROCESSING PLAYER https://www.basketball-r

PROCESSING PLAYER https://www.basketball-reference.com/players/p/petruri01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/w/whitnch01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/t/thompke01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/t/thompke01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/t/thompke01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/t/thompke01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/p/peplomi01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/p/peplomi01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/p/peplomi01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/r/robingl01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/k/kiddja01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/h/hillgr01.html
PROCESSING PLAYER https://www.basketball-r

PROCESSING PLAYER https://www.basketball-reference.com/players/b/boycedo01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/snower01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/snower01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/snower01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/snower01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/e/edneyty01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/d/davisma02.html
PROCESSING PLAYER https://www.basketball-reference.com/players/a/allenje01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/l/lewisma01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/l/lewisma01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/h/hoibefr01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/h/hoibefr01.html
PROCESSING PLAYER https://www.basketball-ref

PROCESSING PLAYER https://www.basketball-reference.com/players/o/obannch01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/c/cottoja02.html
PROCESSING PLAYER https://www.basketball-reference.com/players/m/milicma01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/w/wellsbu01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/stewake01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/c/collija01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/j/jacksma02.html
PROCESSING PLAYER https://www.basketball-reference.com/players/h/honeyje01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/j/johnsan02.html
PROCESSING PLAYER https://www.basketball-reference.com/players/j/johnsan02.html
PROCESSING PLAYER https://www.basketball-reference.com/players/l/lawsoja01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/j/jacksst02.html
PROCESSING PLAYER https://www.basketball

PROCESSING PLAYER https://www.basketball-reference.com/players/l/lewisqu01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/g/glovedi01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/f/fosteje01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/t/thomake01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/g/georgde01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/k/kirilan01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/j/jamesti01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/c/cummivo01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/j/jonesju01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/p/padgesc01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/smithle01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/c/celesjo01.html
PROCESSING PLAYER https://www.basketball

PROCESSING PLAYER https://www.basketball-reference.com/players/b/battish01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/g/griffed01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/d/diopde01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/w/whitero02.html
PROCESSING PLAYER https://www.basketball-reference.com/players/j/johnsjo02.html
PROCESSING PLAYER https://www.basketball-reference.com/players/b/brownke01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/r/radmavl01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/j/jefferi01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/m/murphtr01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/h/huntest01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/h/hastoki01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/b/bradlmi01.html
PROCESSING PLAYER https://www.basketball-

PROCESSING PLAYER https://www.basketball-reference.com/players/b/butlera01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/slayta01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/slayta01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/s/scolalu01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/h/holcora01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/e/edwarco01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/j/jamesle01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/m/milicda01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/a/anthoca01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/b/boshch01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/w/wadedw01.html
PROCESSING PLAYER https://www.basketball-reference.com/players/k/kamanch01.html
PROCESSING PLAYER https://www.basketball-ref